#  Преобразование текстов в последовательность индексов токенов. 🤗 Tokenizers

__Автор задач: Блохин Н.В. (NVBlokhin@fa.ru)__

Материалы:
* Deep Learning with PyTorch (2020) Авторы: Eli Stevens, Luca Antiga, Thomas Viehmann
* https://huggingface.co/docs/tokenizers/index
* https://huggingface.co/docs/tokenizers/pipeline
* https://huggingface.co/docs/tokenizers/api/trainers#tokenizers.trainers.WordLevelTrainer
* Хороший минималистичный пакет с набором готовых преобразований, но больше не развивается:
    * https://pytorch.org/text/stable/
    * https://pytorch.org/text/stable/vocab.html
    * https://pytorch.org/text/stable/transforms.html

## Задачи для совместного разбора

1\. Рассмотрите основные шаги по преобразованию текста в последовательность индексов токенов.

In [ ]:
corpus = [
    "Студенты усердно занимаются стремясь получить знания и достичь успеха",
    "Студенты активно участвуют в общественной жизни университета"
]

In [ ]:
import nltk
from nltk.tokenize import word_tokenize
nltk.download('punkt_tab')
word_tokenize("")

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


[]

In [ ]:
corpus_tokens = [
    word_tokenize(doc.lower())
    for doc in corpus
]

In [ ]:
corpus_tokens

[['студенты',
  'усердно',
  'занимаются',
  'стремясь',
  'получить',
  'знания',
  'и',
  'достичь',
  'успеха'],
 ['студенты',
  'активно',
  'участвуют',
  'в',
  'общественной',
  'жизни',
  'университета']]

In [ ]:
words = set()
words.update(corpus_tokens[0])
words.update(corpus_tokens[1])
words = list(words)

In [ ]:
stoi = {w: idx for idx, w in enumerate(words)}
stoi

{'стремясь': 0,
 'активно': 1,
 'в': 2,
 'занимаются': 3,
 'университета': 4,
 'студенты': 5,
 'получить': 6,
 'достичь': 7,
 'усердно': 8,
 'участвуют': 9,
 'жизни': 10,
 'успеха': 11,
 'общественной': 12,
 'знания': 13,
 'и': 14}

In [ ]:
corpus_i = [
    [stoi[w] for w in doc]
    for doc in corpus_tokens
]
corpus_i

[[5, 8, 3, 0, 6, 13, 14, 7, 11], [5, 1, 9, 2, 12, 10, 4]]

In [ ]:
import torch as th

th.tensor(corpus_i[0]).long()

tensor([ 5,  8,  3,  0,  6, 13, 14,  7, 11])

In [ ]:
# stoi["никита"]
stoi.get("никита", "<UNK>")

'<UNK>'

In [ ]:
th.tensor(corpus_i)

ValueError: expected sequence of length 9 at dim 1 (got 7)

In [ ]:
stoi.get("никита", "<PAD>") # padding

'<PAD>'

In [ ]:
stoi["<PAD>"] = len(stoi)
stoi["<UNK>"] = len(stoi)

In [ ]:
corpus_i[1].extend((stoi["<PAD>"], stoi["<PAD>"]))

In [ ]:
th.tensor(corpus_i)

tensor([[ 5,  8,  3,  0,  6, 13, 14,  7, 11],
        [ 5,  1,  9,  2, 12, 10,  4, 15, 15]])

2\. Обсудите основные требования к инструменту для построения набора индексов

3\. Рассмотрите пример работы с пакетом `tokenizers` для построения набора индексов токенов.

In [ ]:
from tokenizers import Tokenizer
from tokenizers.models import WordLevel
from tokenizers.trainers import WordLevelTrainer
from tokenizers.pre_tokenizers import Whitespace
from tokenizers.normalizers import Lowercase, Replace, Sequence

In [ ]:
corpus = [
    "Маленький котенок игриво прыгает за шариком",
    "Пушистый котик мурлыкает, лежа на солнышке",
    "Котенок любопытно нюхает цветы в саду",
    "Котик ловко лазает по дереву, исследуя окружающий мир",
    "Спящий котик мило моргает своими яркими глазками",
]

In [ ]:
tokenizer = Tokenizer(WordLevel())

tokenizer.normalizer = Sequence([Lowercase(), Replace(",", " ")])
tokenizer.pre_tokenizer = Whitespace()

trainer = WordLevelTrainer(special_tokens=["<PAD>", "<UNK>"])
tokenizer.train_from_iterator(corpus, trainer=trainer)

In [ ]:
tokenizer.get_vocab()

{'по': 22,
 'исследуя': 9,
 'на': 19,
 'цветы': 29,
 'котик': 2,
 'мило': 15,
 'игриво': 8,
 'маленький': 14,
 'мурлыкает': 18,
 'в': 4,
 'лежа': 11,
 'ловко': 12,
 'спящий': 28,
 'глазками': 5,
 'прыгает': 23,
 'за': 7,
 'саду': 25,
 'лазает': 10,
 'нюхает': 20,
 'моргает': 17,
 'своими': 26,
 'яркими': 31,
 'шариком': 30,
 '<UNK>': 1,
 'любопытно': 13,
 'окружающий': 21,
 'солнышке': 27,
 'мир': 16,
 '<PAD>': 0,
 'пушистый': 24,
 'дереву': 6,
 'котенок': 3}

In [ ]:
enc = tokenizer.encode(corpus[0])
enc.tokens

['маленький', 'котенок', 'игриво', 'прыгает', 'за', 'шариком']

In [ ]:
enc.ids

[14, 3, 8, 23, 7, 30]

## Задачи для самостоятельного решения

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


<p class="task" id="1"></p>

1\. Опишите класс `Vocab`. При создании объекта `Vocab` в конструктор передается набор текстов, предварительно разбитых на токены. Объект должен позволять:
* по токену получить его уникальный индекс (в случае отсутствия токена в словаре вернуть 1)
* по индексу токена получить сам токен (в случае отсутствия токена в словаре вернуть <UNK>)

Первые 4 индекса зарезервированы под специальные токены `<PAD>`, `<UNK>`, `<SOS>`, `<EOS>`.
    
Создайте `Vocab` на основе списка `corpus` и закодируйте каждый токен в предложениях, используя `Vocab`. Выведите полученный результат на экран.

- [ ] Проверено на семинаре

In [ ]:
word_tokenize('Привет,  ...  номер 7')

['Привет', ',', '...', 'номер', '7']

In [ ]:
from nltk.tokenize import word_tokenize


class Vocab():
    def __init__(self, data):
        uniq = '<PAD>, <UNK>, <SOS>, <EOS>'.split(', ')
        corpus = dict()
        for i in range(len(uniq)):
            # print(i, uniq[i])
            corpus[uniq[i]] = i
        k = len(uniq)
        for row in data:
            for word in word_tokenize(row.lower()):
                if word not in corpus:
                    corpus[word] = k
                    k += 1
        self.corpus = corpus

        inverse_corpus = dict()
        for k, v in corpus.items():
            inverse_corpus[v] = k

        self.inverse_corpus = inverse_corpus


    def itos(self, idx):
        """Возвращает токен по индексу"""
        return self.inverse_corpus.get(idx, '<UNK>')

    def stoi(self, s):
        """Возвращает индекс токена"""

        return self.corpus.get(s, 1)

In [ ]:
data = [
    "Маленький котенок игриво прыгает за шариком",
    "Пушистый котик мурлыкает, лежа на солнышке",
    "Котенок любопытно нюхает цветы в саду",
    "Котик ловко лазает по дереву, исследуя окружающий мир",
    "Спящий котик мило моргает своими яркими глазками",
]

In [ ]:
n1 = Vocab(data)

In [ ]:
n1.itos(10)

'пушистый'

In [ ]:
n1.stoi('нюхает')

18

In [ ]:
n1.corpus

{'<PAD>': 0,
 '<UNK>': 1,
 '<SOS>': 2,
 '<EOS>': 3,
 'маленький': 4,
 'котенок': 5,
 'игриво': 6,
 'прыгает': 7,
 'за': 8,
 'шариком': 9,
 'пушистый': 10,
 'котик': 11,
 'мурлыкает': 12,
 ',': 13,
 'лежа': 14,
 'на': 15,
 'солнышке': 16,
 'любопытно': 17,
 'нюхает': 18,
 'цветы': 19,
 'в': 20,
 'саду': 21,
 'ловко': 22,
 'лазает': 23,
 'по': 24,
 'дереву': 25,
 'исследуя': 26,
 'окружающий': 27,
 'мир': 28,
 'спящий': 29,
 'мило': 30,
 'моргает': 31,
 'своими': 32,
 'яркими': 33,
 'глазками': 34}

In [ ]:
n1.inverse_corpus

{0: '<PAD>',
 1: '<UNK>',
 2: '<SOS>',
 3: '<EOS>',
 4: 'маленький',
 5: 'котенок',
 6: 'игриво',
 7: 'прыгает',
 8: 'за',
 9: 'шариком',
 10: 'пушистый',
 11: 'котик',
 12: 'мурлыкает',
 13: ',',
 14: 'лежа',
 15: 'на',
 16: 'солнышке',
 17: 'любопытно',
 18: 'нюхает',
 19: 'цветы',
 20: 'в',
 21: 'саду',
 22: 'ловко',
 23: 'лазает',
 24: 'по',
 25: 'дереву',
 26: 'исследуя',
 27: 'окружающий',
 28: 'мир',
 29: 'спящий',
 30: 'мило',
 31: 'моргает',
 32: 'своими',
 33: 'яркими',
 34: 'глазками'}

<p class="task" id="2"></p>

2\. Создайте класс `NewsDataset` на основе данных из файла `news.csv`. Реализуйте метод `__getitem__` таким образом, чтобы он возвращал набор индексов токенов для заголовка новости $i$ и метку класса для этой новости. Для кодирования текстов используйте собственную реализацию `Vocab`. Набор индексов токенов возвращайте в виде тензора. Предусмотрите возможность передавать в конструктор класса преобразования.

Выведите на экран результат выполнения `dataset[0]`.

- [ ] Проверено на семинаре

In [ ]:
import pandas as pd
import torch
df = pd.read_csv('/content/drive/MyDrive/NLP/4/news.csv')
df

,text,label
0,лукашенко пригрозил литовским танкам белорусск...,0
1,российские компании оказались в опасности из з...,1
2,лукашенко объявил об отмене выборов из за напа...,0
3,«роснефть» стала лидером по объему биржевых пр...,1
4,псковской области дадут десятки миллионов рубл...,1
...,...,...
1096,борис джонсон пообещал избирателям вернуть ирл...,0
1097,михаил горбачев заявил что не подавал в отстав...,0
1098,в мгимо появится факультет международных отнош...,0
1099,в россии введут штраф за отрицание агрессии нато,0


In [ ]:
N = df.shape[0]
N

1101

In [ ]:
for row in range(N):
    print(df.iloc(row)[0]['text'])
    break

лукашенко пригрозил литовским танкам белорусской картошкой 


In [ ]:
for ind, (t, l) in df.iterrows():
    print(t, l)
    # print(t[1])
    break

лукашенко пригрозил литовским танкам белорусской картошкой  0


In [ ]:
a = Vocab(df['text'])
len(a.corpus), a.corpus

(5996,
 {'<PAD>': 0,
  '<UNK>': 1,
  '<SOS>': 2,
  '<EOS>': 3,
  'лукашенко': 4,
  'пригрозил': 5,
  'литовским': 6,
  'танкам': 7,
  'белорусской': 8,
  'картошкой': 9,
  'российские': 10,
  'компании': 11,
  'оказались': 12,
  'в': 13,
  'опасности': 14,
  'из': 15,
  'за': 16,
  'глобального': 17,
  'потепления': 18,
  'объявил': 19,
  'об': 20,
  'отмене': 21,
  'выборов': 22,
  'нападения': 23,
  'иностранных': 24,
  'диверсантов': 25,
  '«': 26,
  'роснефть': 27,
  '»': 28,
  'стала': 29,
  'лидером': 30,
  'по': 31,
  'объему': 32,
  'биржевых': 33,
  'продаж': 34,
  'нефтепродуктов': 35,
  'псковской': 36,
  'области': 37,
  'дадут': 38,
  'десятки': 39,
  'миллионов': 40,
  'рублей': 41,
  'на': 42,
  'ремонт': 43,
  'дорог': 44,
  'компания': 45,
  'henkel': 46,
  '—': 47,
  'владелец': 48,
  'брендов': 49,
  'schwarzkopf': 50,
  'persil': 51,
  'и': 52,
  'момент': 53,
  'объявила': 54,
  'уходе': 55,
  'россии': 56,
  'чубайс': 57,
  'ушел': 58,
  'совета': 59,
  'директоро

In [ ]:
class NewsDataset():
    def __init__(self, data, pad=None):
        N = data.shape[0]
        base = []


        data['text'] = data['text'].str.replace(',',' ').replace('.',' ').replace('!',' ').replace('?',' ')

        tokens = Vocab(data['text'])
        for idx, (text, label) in data.iterrows():
            base.append([label, torch.tensor([tokens.corpus[i] for i in word_tokenize(text)])])


        if pad != None:
            base = pad.get(base)

        self.base= base
        self.tokens= tokens


    def __getitem__(self, idx):
        return self.base[idx]


    def get_text(self):
        return [x[1] for x in self.base]

In [ ]:
data = NewsDataset(df)

In [ ]:
data[0][1][:5]

tensor([4, 5, 6, 7, 8])

In [ ]:
data.get_text()

[tensor([4, 5, 6, 7, 8, 9]),
 tensor([10, 11, 12, 13, 14, 15, 16, 17, 18]),
 tensor([ 4, 19, 20, 21, 22, 15, 16, 23, 24, 25]),
 tensor([26, 27, 28, 29, 30, 31, 32, 33, 34, 35]),
 tensor([36, 37, 38, 39, 40, 41, 42, 43, 44]),
 tensor([45, 46, 47, 48, 49, 50, 51, 52, 26, 53, 28, 47, 54, 20, 55, 15, 56]),
 tensor([57, 58, 15, 59, 60, 61, 26, 62, 28, 63, 64, 65, 66, 15, 56]),
 tensor([67, 68, 69, 26, 70, 71, 72, 15, 73, 74, 16, 75, 52, 76, 28]),
 tensor([77, 78, 79, 80, 81, 82, 83, 84]),
 tensor([85, 86, 87, 88, 89, 42, 90, 91, 92, 15, 16, 93, 94]),
 tensor([ 10,  95,  96,  97,  98,  99, 100, 101, 102, 103,  41, 104,  16, 105,
         106, 107, 108]),
 tensor([ 56, 109, 110, 111, 112]),
 tensor([113, 114, 115, 116, 117, 118, 119, 120, 121, 122,  13, 123]),
 tensor([ 42, 124,  13, 125, 126, 127, 128, 129]),
 tensor([130, 131, 132, 133, 134, 135, 136,  13, 137]),
 tensor([138, 139, 140, 141, 142,  42, 143]),
 tensor([ 26, 144, 145, 146, 147, 148, 149, 150,  13, 151,  28, 152, 153, 154,
    

In [ ]:
a = torch.tensor([1, 2, 3])
a

tensor([1, 2, 3])

<p class="task" id="3"></p>

3\. Реализуйте преобразование `Truncate`, которое обрезает каждый текст в батче до `n` токенов. Создайте объект `NewsDataset` с обрезкой предложений до 5 токенов (указав данное преобразование при создании объекта). Создайте батч из 16 примеров при помощи стандартного `torch.utils.data.DataLoader`. Выведите на экран батч и размеры его компонент.

- [ ] Проверено на семинаре

In [4]:
from torch.utils.data import DataLoader
from torch.nn.functional import pad
import torch

In [5]:
a = torch.tensor([1, 2, 3])

In [6]:
pad(a, (0, 5))

tensor([1, 2, 3, 0, 0, 0, 0, 0])

In [ ]:
class Truncate:
    def __init__(self, data, N):
        N_data = []
        for row in data:
            n = len(row)
            if n > N:
                N_data.append(row[:N])
            else:
                N_data.append(pad(row, (0, N-n)))
        self.data = N_data

    def batches(self, batch_size):
        return DataLoader(self.data, batch_size=batch_size)

In [ ]:
test1 = Truncate(data.get_text(), 5)
batch = test1.batches(16)

In [ ]:
for idx, x in enumerate(batch):
    print(x)

tensor([[  4,   5,   6,   7,   8],
        [ 10,  11,  12,  13,  14],
        [  4,  19,  20,  21,  22],
        [ 26,  27,  28,  29,  30],
        [ 36,  37,  38,  39,  40],
        [ 45,  46,  47,  48,  49],
        [ 57,  58,  15,  59,  60],
        [ 67,  68,  69,  26,  70],
        [ 77,  78,  79,  80,  81],
        [ 85,  86,  87,  88,  89],
        [ 10,  95,  96,  97,  98],
        [ 56, 109, 110, 111, 112],
        [113, 114, 115, 116, 117],
        [ 42, 124,  13, 125, 126],
        [130, 131, 132, 133, 134],
        [138, 139, 140, 141, 142]])
tensor([[ 26, 144, 145, 146, 147],
        [160,  13, 161, 162, 163],
        [ 13, 174, 175, 176, 177],
        [178, 179, 180, 181,  76],
        [ 13, 187,  13, 188, 189],
        [ 26, 192, 193, 194, 195],
        [205, 206, 207, 208,  31],
        [215, 216, 217,  13, 218],
        [227, 228, 229,  15, 230],
        [231,  42, 124,  99, 232],
        [239,  26, 240,  28, 241],
        [ 13, 249, 250, 251,  13],
        [259, 260, 

<p class="task" id="4"></p>

4\. Реализуйте преобразование `Pad`, которое расширяет каждый текст в батче до `n` токенов значением `pad_idx`. Создайте объект `NewsDataset` с расширением предложений до 30 токенов (указав данное преобразование при создании объекта). Создайте батч из 16 примеров при помощи стандартного `torch.utils.data.DataLoader`. Выведите на экран батч и размеры его компонент.

- [ ] Проверено на семинаре

функция уже реализована в 3 номере внутри Truncate, попробуем реализовать ее внутри

In [ ]:
[0] +[1]*2

[0, 1, 1]

In [ ]:
class Pad:
    def __init__(self, n, pad_idx):
        self.n = n
        self.pad_idx = pad_idx

    def get(self, data):
        N_data = []
        for row in data:
            n = len(row)
            if n < self.n:
                N_data.append(row + [self.pad_idx]*(self.n - n))
            if n >= self.n:
                N_data.append(row[:self.n])
        return N_data

In [ ]:
data = Truncate(NewsDataset(df, Pad(30, 0)).get_text(), 30)
batch = data.batches(16)

for idx, x in enumerate(batch):
    print(x)
    print(x.shape)
    break


tensor([[  4,   5,   6,   7,   8,   9,   0,   0,   0,   0,   0,   0,   0,   0,
           0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
           0,   0],
        [ 10,  11,  12,  13,  14,  15,  16,  17,  18,   0,   0,   0,   0,   0,
           0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
           0,   0],
        [  4,  19,  20,  21,  22,  15,  16,  23,  24,  25,   0,   0,   0,   0,
           0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
           0,   0],
        [ 26,  27,  28,  29,  30,  31,  32,  33,  34,  35,   0,   0,   0,   0,
           0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
           0,   0],
        [ 36,  37,  38,  39,  40,  41,  42,  43,  44,   0,   0,   0,   0,   0,
           0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
           0,   0],
        [ 45,  46,  47,  48,  49,  50,  51,  52,  26,  53,  28,  47,  54,  20,
          55,  15,  56,   0,   

<p class="task" id="5"></p>

5\. Создайте объект `tokenizers.Tokenizer` на основе данных из файла `news.csv`. Для выделения токенов из текста используйте разбиение по пробелам (pre-tokenizer `Whitespace` + model `WordLevel` ). На этапе нормализации приводите текст к нижнему регистру и убирайте знаки препинания. Включите опцию паддинга. Первые 4 индекса зарезервируйте под специальные токены `<PAD>`, `<UNK>`, `<SOS>`, `<EOS>`.

Опишите класс `NewsDatasetHfTokenizer`. Реализуйте метод `__getitem__` таким образом, чтобы он возвращал набор индексов токенов для заголовка новости $i$ и метку класса для этой новости. Для кодирования текстов используйте обученный токенизатор. Набор индексов токенов возвращайте в виде тензора. Создайте батч из 16 примеров при помощи стандартного `torch.utils.data.DataLoader`. Выведите на экран батч и размеры его компонент.

- [ ] Проверено на семинаре

-------------

*Так как для создания батча тензеров нужна одинаковая размерность тензеров, то будем заполнять каждую строку до 30 токенов нулями (pad)*

In [ ]:
from tokenizers import Tokenizer
from tokenizers.models import WordLevel
from tokenizers.trainers import WordLevelTrainer
from tokenizers.pre_tokenizers import Whitespace
from tokenizers.normalizers import Lowercase, Replace, Sequence

In [ ]:
tokenizer = Tokenizer(WordLevel())

tokenizer.normalizer = Sequence([Lowercase(), Replace(",", " "), Replace('.', ' '), Replace('!', ' '), Replace('?', ' ')])
tokenizer.pre_tokenizer = Whitespace()

trainer = WordLevelTrainer(special_tokens=["<PAD>", "<UNK>", '<SOS>', '<EOS>'])
tokenizer.train_from_iterator(df['text'], trainer=trainer)

In [ ]:
tokenizer.get_vocab_size()

5996

In [ ]:
tokenizer.get_vocab()['<PAD>']

0

In [ ]:
a = tokenizer.encode(df['text'][0]).ids
a

[31, 203, 3356, 5415, 491, 3096]

In [ ]:
from torch.nn.functional import pad
pad(torch.tensor([1, 2, 3]), (0, 3))

tensor([1, 2, 3, 0, 0, 0])

In [ ]:


class NewsDatasetHfTokenizer():
    def __init__(self, data):
        base = []
        text_only = []

        data['text'] = data['text'].str.replace(',',' ').replace('.',' ').replace('!',' ').replace('?',' ')


        for idx, (text, label) in data.iterrows():
            x = torch.tensor(tokenizer.encode(text).ids)
            n = len(x)
            if n < 30:
                x = pad(x, (0, 30-n))
            elif n > 30:
                x = x[:30]
            base.append([label, x])
            text_only.append(x)

        self.base= base
        self.text_only= text_only


    def __getitem__(self, idx):
        return self.base[idx]

    def get_text(self):
        return self.text_only

    def batches(self, batch_size):
        return DataLoader(self.text_only, batch_size=batch_size)



In [ ]:
data = NewsDatasetHfTokenizer(df)

In [ ]:
batch = data.batches(16)
for idx, x in enumerate(batch):
    print(x)
    print(x.shape)
    break

tensor([[  31,  203, 3356, 5415,  491, 3096,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0],
        [  27,  147, 3914,    4,  379,   12,   10, 2425, 4486,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0],
        [  31,   64,   29, 4037,  318,   12,   10, 3668,  568,  993,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0],
        [   5, 4957,    6,  728, 1129,   14, 3872, 1979, 4683, 3777,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0],
        [4767,   49,  971,  984,  189,   65,    7, 4917, 2652,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
      